## JEM207 Data Processing in Python - Midterm
[Daniel Brosko](mailto:89227653@fsv.cuni.cz), April 5th, 2022

In [71]:
#%matplotlib inline
import matplotlib.pyplot as plt

import requests
import json
import re

import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

#from IPython.core.display import HTML, display

## Task 1

In [72]:
req = requests.post("https://ies-midterm.soulution.rocks/login", json ={"cuni": "89227653"})

response = req.json()
print(response)

{'data': {'personal_code': '4246f93a', 'dataset_ids': ['24f88cb68c', '484090f05d', '557b58dd1d', '3e8641892a', '11f651d1ee', '0a42bff0a1']}, 'message': 'Hello, Broško Daniel, Bc! Good luck with the midterm. GET your data set from /data/file_id endpoint; use the codes you received in the data response as the file_id parameter! Print the personal code in your notebook.'}


In [73]:
print('Hello, my personal code is:', response['data']['personal_code'])

Hello, my personal code is: 4246f93a


In [74]:
my_codes = response['data']['dataset_ids']
print('Codes are:', my_codes)

Codes are: ['24f88cb68c', '484090f05d', '557b58dd1d', '3e8641892a', '11f651d1ee', '0a42bff0a1']


In [76]:
my_data = list()

for j in my_codes:
    req = requests.get(f"https://ies-midterm.soulution.rocks/data/{j}")
    #print(req)
    if req.status_code == 200: 
        res = req.json()
        my_data.append(res)
    else:
        print(req.status_code,'Error when dowloading code', j)
        while req.status_code != 200:
            
            req = requests.get(f"https://ies-midterm.soulution.rocks/data/{j}")
            print(req)
            res = req.json()
            my_data.append(res)
    
print(my_data)

503 Error when dowloading code 24f88cb68c
<Response [200]>
503 Error when dowloading code 484090f05d
<Response [200]>
503 Error when dowloading code 11f651d1ee
<Response [200]>
[{'data': {'company': 'PSN', 'data': [['2020-05-06', '36.0', '37.209999084472656', '34.72999954223633', '35.9900016784668', '35.9900016784668', '380000'], ['2020-09-23', '34.310001373291016', '34.47999954223633', '33.70000076293945', '34.0', '34.0', '966100'], ['2019-09-20', '37.369998931884766', '37.400001525878906', '34.93000030517578', '34.93000030517578', '34.93000030517578', '2302100'], ['2020-02-19', '44.880001068115234', '45.400001525878906', '44.630001068115234', '45.029998779296875', '45.029998779296875', '88800'], ['2019-07-16', '38.0', '38.0', '37.130001068115234', '37.33000183105469', '37.33000183105469', '180100'], ['2019-07-19', '36.2400016784668', '37.31999969482422', '35.68000030517578', '36.84000015258789', '36.84000015258789', '269400'], ['2019-06-18', '32.43000030517578', '33.880001068115234',

In [98]:
#'Columns are in order Date,Open,High,Low,Close,Adj.Close,Volume. Make sure your data are ordered.'},
DF = pd.DataFrame()
company = []

for j in range(0, len(my_data)):
    df = pd.DataFrame(my_data[j]['data'])
    df['company'] = my_data[j]['data']['company']
    df['Date'] = my_data[j]['data']['data'][0][0]
    df['Open'] = my_data[j]['data']['data'][0][1]
    df['High'] = my_data[j]['data']['data'][0][2]
    df['Low'] = my_data[j]['data']['data'][0][3]
    df['Close'] = my_data[j]['data']['data'][0][4]
    df['Adj. Close'] = my_data[j]['data']['data'][0][5]
    df['Volume'] = my_data[j]['data']['data'][0][6]
    df.drop('data',1, inplace=True)
    company.append(my_data[j]['data']['company'])
    
    DF = pd.concat([DF,df], axis= 0)
    
DF = DF.dropna()
DF = DF.sort_index()

print(DF)

    company        Date                 Open                 High  \
0       PSN  2020-05-06                 36.0   37.209999084472656   
0       AZN  2019-01-02   37.720001220703125                 38.0   
0       BHP  2019-01-02    46.63999938964844   47.900001525878906   
0       MNG  2019-01-02   35.630001068115234   36.810001373291016   
0      SGRO  2019-03-05  0.20999999344348907  0.20999999344348907   
..      ...         ...                  ...                  ...   
477     BHP  2019-01-02    46.63999938964844   47.900001525878906   
478     MNG  2019-01-02   35.630001068115234   36.810001373291016   
478     BHP  2019-01-02    46.63999938964844   47.900001525878906   
478     AZN  2019-01-02   37.720001220703125                 38.0   
478      HL  2019-01-02   2.3399999141693115                  2.5   

                     Low                Close           Adj. Close   Volume  
0      34.72999954223633     35.9900016784668     35.9900016784668   380000  
0      37.59000

## Task 2